Executar um script de treinamento com SDK do Python

In [1]:
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token sucessfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines 

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

ValidationException: We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.

In [ ]:
%%writefile src/sorvetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import logisticRegression

# load the sorvetes dataset
print("Loding Data...")
Sorvetes = pd.read_csv('sorvetes.csv')

#separate features and labels
x,y = sorvetes[['Temperatura']].values, sorvetes['Vendas'].values

# split data into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = logisticRegression(C=1/reg, solver="liblinear").fit(x_train, y_train)

# calculate accuracy
y_hat = model.predict(x_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(x_test)
auc = roc_auc_score(y_test, y_scores[:,1]
print('AUC: ' + str(auc)))

Writing src/sorvetes-training.py


In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-instancias",
    display_name="sorvetes-python-train",
    experiment_name="sorvetes-training"
)

#submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)